In [1]:
#data manipulation
import numpy as np
import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
import nltk
from nltk.tokenize import word_tokenize

#stats/ml
import scipy
import sklearn as sk
from scipy.stats import ttest_1samp
from scipy.stats import boxcox
from scipy.stats import chisquare
from sklearn.preprocessing import OneHotEncoder
#vis
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

#additional support packages
from datetime import date, datetime, timedelta
import random
import re
import os
# nltk.download('punkt')
os.cpu_count()

16

In [2]:
def bin_small(col,replace_val,threshold=100,**kwargs):
    more_removal = [i for i,x in dict(data[col].value_counts()).items() if x<threshold]
    data[col]=np.where(data[col].isin(more_removal),replace_val,data[col])

<a id='top'></a>
# Overview

1. [Data Cleaning](#sec-1)
- Nulls
- Data inconsistencies
    
    
2. [Feature Engineering](#sec-2)
- 5 new features


3. [Additional Exploratory Data Analysis (EDA)](#sec-3)
- Univariate analysis of dependent variable
- Exploring independent variables relationships with each other and dependent variable

<a id='sec-1'></a>
## 1. Data Cleaning

In [3]:
data = pd.read_csv(r'..\data\adopt.csv')
legacy_data=data.copy()
data

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
0,A745079,Transfer,Unknown,NaN,7920 Old Lockhart in Travis (TX),Stray,Normal,Cat,Unknown,Domestic Shorthair Mix,Blue,3,0
1,A801765,Transfer,Intact Female,NaN,5006 Table Top in Austin (TX),Stray,Normal,Cat,Intact Female,Domestic Shorthair,Brown Tabby/White,28,0
2,A667965,Transfer,Neutered Male,NaN,14100 Thermal Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,Chihuahua Shorthair Mix,Brown/Tan,1825,0
3,A687551,Transfer,Intact Male,NaN,5811 Cedardale Dr in Austin (TX),Stray,Normal,Cat,Intact Male,Domestic Shorthair Mix,Brown Tabby,28,0
4,A773004,Adoption,Neutered Male,*Boris,Highway 290 And Arterial A in Austin (TX),Stray,Normal,Dog,Intact Male,Chihuahua Shorthair Mix,Tricolor/Cream,365,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95379,A706720,Adoption,Neutered Male,Nikko,Mc Callen Pass And Parmer in Austin (TX),Stray,Normal,Dog,Neutered Male,Miniature Schnauzer Mix,Tan/Gray,1460,0
95380,A782751,Adoption,Neutered Male,NaN,18706 Blake Manor Rd in Manor (TX),Stray,Normal,Dog,Intact Male,American Pit Bull Terrier Mix,Brown,60,0
95381,A768058,Euthanasia,Unknown,NaN,1701 Congress Avenue in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Black/Black,730,0
95382,A729326,Adoption,Neutered Male,*Jester,5017 W. 290 in Austin (TX),Stray,Normal,Dog,Intact Male,Pointer Mix,Black/White,730,0


In [4]:
data['Outcome Type'].value_counts()

Adoption           39734
Transfer           32299
Return to Owner    13719
Euthanasia          7630
Died                1053
Disposal             503
Rto-Adopt            369
Missing               52
Relocate              20
Name: Outcome Type, dtype: int64

In [5]:
data['Time at Center'].value_counts()[1]/data['Time at Center'].value_counts()[0]

0.09577584523303502

In [6]:
data['Time at Center'].value_counts()

0    87047
1     8337
Name: Time at Center, dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95384 entries, 0 to 95383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Pet ID                95384 non-null  object
 1   Outcome Type          95379 non-null  object
 2   Sex upon Outcome      95383 non-null  object
 3   Name                  59093 non-null  object
 4   Found Location        95384 non-null  object
 5   Intake Type           95384 non-null  object
 6   Intake Condition      95384 non-null  object
 7   Pet Type              95384 non-null  object
 8   Sex upon Intake       95383 non-null  object
 9   Breed                 95384 non-null  object
 10  Color                 95384 non-null  object
 11  Age upon Intake Days  95384 non-null  int64 
 12  Time at Center        95384 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 9.5+ MB


### KEY POINTS IDENTIFIED:
    
1. **Primary key - "Pet ID" column**
    - We will check for uniqueness later


1. **4 columns with inconsistent non-null counts**
    - Name - 59093 count
    - Outcome - 95379 count
    - Sex upon Outcome - 95383 count
    - Sex upon Intake - 95383 count
    
    
3. **One integer, independent variable**
    - Age (days), discrete values 
    
    
4. **10 objects, independent variables**
    - Categorical
    - *Note: "Name" weak as categorical. Given the high nulls and high uniqueness of names, it is likely we may have to drop name as a feature, will explore further*
    
    
5. **Dependent variable: "Time at Center"**

In [8]:
data.describe()

,Age upon Intake Days,Time at Center
count,95384.000000,95384.000000
mean,704.154691,0.087405
std,1052.579518,0.282429
min,0.000000,0.000000
25%,30.000000,0.000000
50%,365.000000,0.000000
75%,730.000000,0.000000
max,9125.000000,1.000000


"Time at Center" can be ignored because it is a boolean masked as an integer column. 

"Age upon Intake Days" the standard deviation is larger than the mean value. This suggests there are some large outliers on the upper bound. We will investigate what steps need to be taken for this column in the section 3 of this notebook

### Checking Primary Key column

In [9]:
data['Pet ID'].value_counts().max()

1

"Pet ID" column is indeed unique. Let's make sure all columns contain the same data type

### Checking for mixed data type columns

In [10]:
for i in data.columns.values:
    type_array = data[i].apply(type).value_counts()
    if len(type_array)>1:
        print(f'{type_array}')
        print()
    else:
        pass

<class 'str'>      95379
<class 'float'>        5
Name: Outcome Type, dtype: int64

<class 'str'>      95383
<class 'float'>        1
Name: Sex upon Outcome, dtype: int64

<class 'str'>      59093
<class 'float'>    36291
Name: Name, dtype: int64

<class 'str'>      95383
<class 'float'>        1
Name: Sex upon Intake, dtype: int64



All columns listed above are columns with null values based on the info method we passed earlier. The additional floating point values are nulls. We're going to take a look at "Outcome Type", "Sex upon Outcome" and "Sex upon Intake" first because the number of null values is low. We should easily be able to determine what to do with these null values. Then we will take a look at "Name"

### Handling "Outcome Type" nulls

In [11]:
data['Outcome Type'].value_counts()

Adoption           39734
Transfer           32299
Return to Owner    13719
Euthanasia          7630
Died                1053
Disposal             503
Rto-Adopt            369
Missing               52
Relocate              20
Name: Outcome Type, dtype: int64

In [12]:
data.loc[data['Outcome Type'].isnull()]

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
828,A674209,NaN,Unknown,NaN,4210 W. Braker in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Brown,0,0
10178,A679066,NaN,Unknown,NaN,Austin (TX),Wildlife,Normal,Other,Unknown,Rabbit Sh,Brown,28,0
18664,A686025,NaN,Unknown,NaN,12901 N Ih 35 #1100 in Austin (TX),Wildlife,Normal,Other,Unknown,Bat Mix,Brown,365,0
47605,A671017,NaN,Unknown,NaN,10105 Falcon Ridge Dr. in Travis (TX),Wildlife,Sick,Other,Unknown,Fox Mix,Brown/Black,365,0
58378,A815725,NaN,Unknown,NaN,321 West Ben White in Austin (TX),Wildlife,Normal,Other,Unknown,Bat,Black,365,0


Based on the values in "Outcome Type" and the number of nulls is low (5), I don't think imputing the values or replacing it with a dummy category like "blank" is necessary and we can likely remove these values. We just need to check these nulls are not associated with rare information in other columns.

Check
- Unknown in "Sex upon Outcome" and "Sex upon Intake" is not associated only with null outcome type
- "Name" has many nulls so we do not have to check
- "Found Location", perhaps City, Province will need to be extracted and checked
- "Intake Type" ensure Wildlife is not associated only with null outcome type
- "Pet Type" ensure Other is not associated with only null outcome type
- "Breed" seems mixed, double check bat info is not under-represented after removal
- "Age upon Intake Days" ensure distribution is not affected from removal of 0,28,365,365,365
- "Time at Center" ensure dependent variable not largely affected by removal

In [13]:
for i in ['Sex upon Outcome','Intake Type','Intake Condition','Sex upon Intake','Pet Type']:
    print(data[i].value_counts())
    print()

Neutered Male    30233
Spayed Female    28123
Intact Female    13686
Intact Male      13614
Unknown           9727
Name: Sex upon Outcome, dtype: int64

Stray                 70112
Owner Surrender       15131
Public Assist          5236
Wildlife               4559
Euthanasia Request      235
Abandoned               111
Name: Intake Type, dtype: int64

Normal      81824
Injured      5384
Sick         4289
Nursing      3163
Aged          352
Other         189
Feral          97
Pregnant       63
Medical        21
Behavior        2
Name: Intake Condition, dtype: int64

Intact Male      33326
Intact Female    32455
Neutered Male    10521
Unknown           9727
Spayed Female     9354
Name: Sex upon Intake, dtype: int64

Dog          48674
Cat          40023
Other         6118
Bird           553
Livestock       16
Name: Pet Type, dtype: int64



In [14]:
data.loc[data.Breed.str.contains('Bat')].shape[0]

3030

Based on these values, removal of the 5 nulls from "Outcome Type" will not impact the dataset. Now to look at "Sex upon Outcome".

In [15]:
data=data.loc[data['Outcome Type'].notnull()] #removing nulls from Outcome Type

### Handling "Sex upon Outcome" and "Sex upon Intake" fields

In [16]:
data.loc[data['Sex upon Outcome'].isnull()] 

,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
38480,A667395,Return to Owner,NaN,Diego,Pflugerville (TX),Owner Surrender,Normal,Dog,NaN,Dachshund,Brown Merle,2555,0


Based on analysis of previous columns, we can include this record by replacing the null values with "Unknown" in both gender related columns. Unknown is already used to fill missing values in these columns.

In [17]:
data['Sex upon Outcome'].fillna('Unknown',inplace=True)
data['Sex upon Intake'].fillna('Unknown',inplace=True)

C:\Users\notal\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Dealing with Name

Because almost 40% of the data has null name values, we will not be removing them. Instead, we will replace the values with "Unknown" to stay consistent with other column nomenclature for null values.


In [18]:
data['Name'].fillna('Unknown',inplace=True)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95379 entries, 0 to 95383
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Pet ID                95379 non-null  object
 1   Outcome Type          95379 non-null  object
 2   Sex upon Outcome      95379 non-null  object
 3   Name                  95379 non-null  object
 4   Found Location        95379 non-null  object
 5   Intake Type           95379 non-null  object
 6   Intake Condition      95379 non-null  object
 7   Pet Type              95379 non-null  object
 8   Sex upon Intake       95379 non-null  object
 9   Breed                 95379 non-null  object
 10  Color                 95379 non-null  object
 11  Age upon Intake Days  95379 non-null  int64 
 12  Time at Center        95379 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 10.2+ MB


Now all columns in our data set have the same number of non-null values. We have removed 5 rows from the entire dataset from this cleaning step

## Ensuring data consistencies
- Noticed "*" prefix to certain names - need to clean this up
- Check other columns for formatting inconsistencies

### "Name" cleaning

In [20]:
len(data.Name.unique())

17464

In [21]:
data['Name']=data['Name'].str.replace('[*]','',regex=True) #remove asteriks
print(len(data.Name.unique()))
data.Name.value_counts()

14107


<ipython-input-21-719e600a3ca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name']=data['Name'].str.replace('[*]','',regex=True) #remove asteriks


Unknown        36304
Bella            370
Max              359
Luna             344
Charlie          305
               ...  
Guidry             1
Roland             1
Von Winkler        1
Wheezer            1
Asher Finn         1
Name: Name, Length: 14107, dtype: int64

In [22]:
legacy_data=data.copy()

Reduced unique values by 2,700. We will skip looking at "Sex upon Outcome", "Intake Type","Intake Condition","Pet Type","Sex upon Intake" because we have seen their values before and they look fine.

We'll now look at "Found Location" column. 

Then "Breed" and "Color".

### "Found Location" check

In [23]:
data['Found Location'].value_counts()

Austin (TX)                                      14821
Travis (TX)                                       1400
7201 Levander Loop in Austin (TX)                  637
Outside Jurisdiction                               605
Del Valle (TX)                                     426
                                                 ...  
5145 Ranch Road 620 North in Travis (TX)             1
2500 Tx-71 in Austin (TX)                            1
S Congress Ave And Alpine Rd E in Austin (TX)        1
800 Norwood Park in Austin (TX)                      1
4509 Stelmo in Austin (TX)                           1
Name: Found Location, Length: 43936, dtype: int64

Based on these values, it seems this column follows a consistent pattern. With 43936 unique values, it will not be a reliable categorical feature unless we can bin together some of these values and create more useful features.

(Street information (Optional)) (City) ((State))

We will look into it a bit further once we decompose it into useful features in the engineering section - City and State. 

### Breed check

In [24]:
data['Breed'].value_counts()

Domestic Shorthair Mix                        27654
Domestic Shorthair                             5068
Pit Bull Mix                                   5013
Chihuahua Shorthair Mix                        4960
Labrador Retriever Mix                         4783
                                              ...  
Siberian Husky/French Bulldog                     1
Toy Fox Terrier/Dachshund                         1
French Bulldog/Chihuahua Shorthair                1
Dachshund/Blue Lacy                               1
American Pit Bull Terrier/American Bulldog        1
Name: Breed, Length: 2395, dtype: int64

2395 unique values is still quite high. The common values are types of mixes and the less common values are also mixes, but in different formats and more detailed.

We can consider strategies to simplify the detailed values that are generating unnecessarily unique values. Additionally we can decompose the "Breed" column into "Breed", which only contains the main breed listed, and "Mix" to define whether the "Breed" is purebred or mixed.

We will also need to look at this at by pet type. Different pet types may have different breed formatting.

In [25]:
data['Pet Type'].value_counts()

Dog          48674
Cat          40023
Other         6113
Bird           553
Livestock       16
Name: Pet Type, dtype: int64

Because the categorical size of  "Livestock" are is small, we will bin it together with "Other". The model will not be able to generalize correctly from such a small bin. "Bird" may be a bit small, but it may be relevant.

In [26]:
data['Pet Type']=data['Pet Type'].replace(to_replace={'Livestock':'Other'})

<ipython-input-26-09d109471959>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Pet Type']=data['Pet Type'].replace(to_replace={'Livestock':'Other'})


In [27]:
data.loc[data['Pet Type']=='Other'].Breed.value_counts()

Bat Mix                1740
Bat                    1287
Raccoon Mix             537
Raccoon                 364
Rabbit Sh Mix           322
                       ... 
Britannia Petit Mix       1
Sheep Mix                 1
Angora-Satin Mix          1
Beveren Mix               1
Otter Mix                 1
Name: Breed, Length: 125, dtype: int64

In [28]:
data.loc[data['Pet Type']=='Bird'].Breed.value_counts()

Chicken Mix      151
Chicken           77
Parakeet Mix      33
Parakeet          25
Duck Mix          21
                ... 
Pheasant Mix       1
Chickadee Mix      1
Finch Mix          1
Catbird Mix        1
Waxwing Mix        1
Name: Breed, Length: 75, dtype: int64

In [29]:
data.loc[data['Pet Type']=='Cat'].Breed.value_counts()

Domestic Shorthair Mix       27654
Domestic Shorthair            5068
Domestic Medium Hair Mix      2784
Domestic Longhair Mix         1359
Siamese Mix                   1132
                             ...  
Domestic Shorthair/Bengal        1
Ocicat Mix                       1
Snowshoe/Siamese                 1
Siamese/Angora                   1
Balinese                         1
Name: Breed, Length: 95, dtype: int64

In [30]:
data.loc[data['Pet Type']=='Dog'].Breed.value_counts()

Pit Bull Mix                                    5013
Chihuahua Shorthair Mix                         4960
Labrador Retriever Mix                          4783
German Shepherd Mix                             2043
Australian Cattle Dog Mix                       1071
                                                ... 
Miniature Schnauzer/Maltese                        1
Catahoula/Vizsla                                   1
Dachshund/Toy Poodle                               1
Australian Shepherd/English Springer Spaniel       1
Black Mouth Cur/Belgian Malinois                   1
Name: Breed, Length: 2101, dtype: int64

After looking at all the different types of animals and their breeds my strategy to improve categorization from these columns is to identify whether a "Breed" value contains "Mix" or "/" and label them as "Mix" in the "Mix" column and as "Not" if otherwise.

We will then extract the string before "Mix" and "/" to re-categorize pure, mix and "mix detailed" formats together.

If that does not solve the complexity of the categorical column, we could use a more complex strategy of identifying the most common pure breed listed instead of the first. This is likely an unnecessary step so I will stick with a more simple strategy and move on.

In [31]:
data['Mix']=np.where((data['Breed'].str.contains('Mix'))|(data['Breed'].str.contains('/')),"Mix","Not")

data['Breed_new']=np.where(data['Breed'].str.contains('Mix'),data['Breed'].str.split('Mix').str[0],
                          np.where(data['Breed'].str.contains('/'),data['Breed'].str.split('/').str[0],data['Breed']))
print(len(data['Breed_new'].unique()))
data['Breed_new']=data['Breed_new'].str.lower()
print(len(data['Breed_new'].unique()))
data['Breed_new']=data['Breed_new'].str.strip()
print(len(data['Breed_new'].unique()))

<ipython-input-31-7c1a223b82dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mix']=np.where((data['Breed'].str.contains('Mix'))|(data['Breed'].str.contains('/')),"Mix","Not")


637
637
362


<ipython-input-31-7c1a223b82dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=np.where(data['Breed'].str.contains('Mix'),data['Breed'].str.split('Mix').str[0],
<ipython-input-31-7c1a223b82dd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=data['Breed_new'].str.lower()
<ipython-input-31-7c1a223b82dd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

Reduced unique values from 2395 to 362. Let's see if there's any further reduction necessary. Categories with only 1-2 values are not good for model generalization on future data

In [32]:
breed=data['Breed_new'].value_counts().to_frame()
breed.describe()

,Breed_new
count,362.000000
mean,263.477901
std,1840.323954
min,1.000000
25%,4.000000
50%,13.000000
75%,69.750000
max,32734.000000


Huge difference between percentiles, mean and max values. More cleanup is required.

In [33]:
breed.sort_values(by='Breed_new',ascending=False).head(20).Breed_new.sum()

74787

In [34]:
breed.loc[breed.Breed_new<5]

,Breed_new
unknown,4
cymric,4
lop-amer fuzzy,4
angora-french,4
muscovy,4
...,...
song bird,1
warbler,1
whimbrel,1
chickadee,1


In [35]:
data['Pet Type'].value_counts()

Dog      48674
Cat      40023
Other     6129
Bird       553
Name: Pet Type, dtype: int64

I was stumped on the best way to reduce the number of bins in this category and just thought of using tokenization to reduce binning further. Other approaches I considered.

- Thresholding by mean or percentile values
    - *Issue: Even binning all categories under 600 in size led to 20k records with undefined breeds - a significant loss of information*
- Thresholding by a proportion of the minority class
    - e.g. Bins must contain 20% of minority class size
- Extract bi-grams to derive smaller groups
- Using Levenshtein distance to organize categories by edit distance
    
    
First we will bin the breeds of smaller "Pet Type" categories like "Bird" and "Other".


In [36]:
for i in data["Pet Type"].unique():
    breed_count = data.loc[data["Pet Type"]==i].Breed_new.value_counts().shape[0]
    print(f'Pet Type: {i}  |  Unique Breeds: {breed_count}')


Pet Type: Cat  |  Unique Breeds: 41
Pet Type: Dog  |  Unique Breeds: 202
Pet Type: Other  |  Unique Breeds: 72
Pet Type: Bird  |  Unique Breeds: 49


In [37]:
data.loc[data["Pet Type"]=='Bird'].Breed_new.value_counts()

chicken         228
parakeet         58
duck             37
dove             28
pigeon           23
cockatiel        18
grackle          15
parrot           11
sparrow          10
peafowl           9
bantam            9
hawk              8
quaker            8
rhode island      6
owl               6
finch             6
lovebird          5
heron             5
mockingbird       5
conure            5
turkey            5
vulture           4
muscovy           4
bluebird          4
cockatoo          4
budgerigar        3
guinea            3
black             2
macaw             2
barred rock       2
crow              2
lark              1
wren              1
bunting           1
song bird         1
warbler           1
african           1
leghorn           1
catbird           1
other bird        1
whimbrel          1
orpington         1
chickadee         1
pheasant          1
nuthatch          1
canary            1
goose             1
silkie            1
waxwing           1
Name: Breed_new, dty

In [38]:
data['Breed_new']=np.where((data['Pet Type']=='Bird')&(data['Breed_new']!='chicken'),'Other Bird',data['Breed_new'])
print(len(data.Breed_new.unique()))

316


<ipython-input-38-6f207744de77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=np.where((data['Pet Type']=='Bird')&(data['Breed_new']!='chicken'),'Other Bird',data['Breed_new'])


In [39]:
for i,x in data.loc[data["Pet Type"]=='Other'].Breed_new.value_counts().iteritems():
    print(f'{i}                {x}')


bat                3027
raccoon                901
rabbit sh                406
guinea pig                335
opossum                320
skunk                178
squirrel                134
rat                109
fox                102
ferret                58
hamster                56
californian                55
snake                34
cold water                30
turtle                29
coyote                25
lionhead                25
rex                22
rabbit lh                17
english spot                16
lop-holland                16
lizard                16
tortoise                14
new zealand wht                13
havana                13
lop-mini                11
lop-english                11
american                10
mouse                10
pig                10
armadillo                10
dutch                9
netherlnd dwarf                9
chinchilla                8
gerbil                7
tarantula                6
rex-mini                5
flemish gian

We will bin everything under 100 count. We could extract rodents to be in a category, as there is enough to make a bin larger than 300. However I don't think its going to affect performance too much.

In [40]:
remove_other = []
for i,x in data.loc[data["Pet Type"]=='Other'].Breed_new.value_counts().iteritems():
    if x<100:
        remove_other.append(i)
    else:
        pass
data['Breed_new']=np.where((data['Pet Type']=='Other')&(data['Breed_new'].isin(remove_other)),'Other',data['Breed_new'])
print(len(data.Breed_new.unique()))

255


<ipython-input-40-615d4d8f5899>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=np.where((data['Pet Type']=='Other')&(data['Breed_new'].isin(remove_other)),'Other',data['Breed_new'])


In [41]:
data.loc[data["Pet Type"]=='Cat'].Breed_new.value_counts()

domestic shorthair         32734
domestic medium hair        3392
domestic longhair           1542
siamese                     1317
american shorthair           240
snowshoe                     181
maine coon                   127
manx                         108
russian blue                  79
ragdoll                       50
himalayan                     44
persian                       28
balinese                      17
bengal                        15
abyssinian                    14
american curl shorthair       13
angora                        13
tonkinese                     12
japanese bobtail               9
turkish van                    8
british shorthair              8
devon rex                      7
turkish angora                 7
bombay                         7
burmese                        6
cornish rex                    5
norwegian forest cat           4
cymric                         4
pixiebob shorthair             4
exotic shorthair               3
havana bro

Just gonna bin anything under 1000.

In [42]:
remove_other = []
for i,x in data.loc[data["Pet Type"]=='Cat'].Breed_new.value_counts().iteritems():
    if x<1000:
        remove_other.append(i)
    else:
        pass
data['Breed_new']=np.where((data['Pet Type']=='Cat')&(data['Breed_new'].isin(remove_other)),'Other Cat',data['Breed_new'])
print(len(data.Breed_new.unique()))

219


<ipython-input-42-253d1cb1b992>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Breed_new']=np.where((data['Pet Type']=='Cat')&(data['Breed_new'].isin(remove_other)),'Other Cat',data['Breed_new'])


In [43]:
for i,x in data.loc[data["Pet Type"]=='Dog'].Breed_new.value_counts().iteritems():
    print(f'{i}                {x}')

labrador retriever                6818
chihuahua shorthair                6411
pit bull                6174
german shepherd                3043
australian cattle dog                1576
dachshund                1416
border collie                982
boxer                933
miniature poodle                908
yorkshire terrier                786
siberian husky                734
australian shepherd                719
beagle                695
great pyrenees                679
miniature schnauzer                655
catahoula                644
rat terrier                637
jack russell terrier                634
shih tzu                593
cairn terrier                570
chihuahua longhair                566
pointer                502
rottweiler                492
anatol shepherd                425
staffordshire                421
plott hound                361
maltese                350
american bulldog                327
pug                309
golden retriever                307
blac

In [44]:
#token approach
breed_dict={}
for i in data.loc[data['Pet Type']=='Dog']['Breed_new'].values:
    salad = word_tokenize(i)
    for s in salad:
        if s in breed_dict.keys():
            breed_dict[s]=breed_dict[s]+1
        else:
            breed_dict[s]=1
breed_list=[[i,x] for i,x in breed_dict.items()]
test=pd.DataFrame(data=breed_list,columns=['Token','Count'])
test.sort_values(by='Count',ascending=False).head(40)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\notal/nltk_data'
    - 'C:\\Users\\notal\\anaconda3\\nltk_data'
    - 'C:\\Users\\notal\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\notal\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\notal\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
d = data.loc[data.Breed_new.str.contains('retriever')].Breed_new.value_counts().iteritems()
next(d)
for i in d:
    print(i)

In [ ]:
replace_dict = {}
for i in test.iterrows():
    if i[1].Count>300:
        val_count = data.loc[(data.Breed_new.str.contains(i[1].Token))&(data['Pet Type']=='Dog')].Breed_new.value_counts()
        if len(val_count)>1:
            iter_obj = val_count.iteritems()
            next(iter_obj)
            for x in iter_obj:
                if x[0] not in replace_dict.keys(): #prevent multiple assignment
                    replace_dict[x[0]]=f'Other {i[1].Token}'
                else:
                    pass
    else:
        val_count = data.loc[(data.Breed_new.str.contains(i[1].Token))&(data['Pet Type']=='Dog')].Breed_new.value_counts()
        iter_obj = val_count.iteritems()
        for x in iter_obj:
            if x[0] not in replace_dict.keys(): #prevent multiple assignment
                replace_dict[x[0]]=f'Other dog'
            else:
                pass
data['Breed_test']=data.Breed_new.replace(to_replace=replace_dict)
# more_removal = [i for i,x in dict(data['Breed_test'].value_counts()).items() if x<300]
# data['Breed_test']=np.where(data['Breed_test'].isin(more_removal),'Other dog',data['Breed_test'])
# data['Breed_test'].value_counts()

In [ ]:
len(data['Breed_test'].unique())

Reduced "Breed" to 47 unique values. We could have reduced it further, but I feel I've done enough.

Let's look at "Color" now

In [ ]:
color=data['Color'].value_counts().to_frame()
color.describe()

Another category with too many bins and too high a varianace

In [ ]:
color.index.values

1. Split mix colors out
2. Identify base color based on first one listed
3. Reduce base categories with too little data points into "Other color"


In [ ]:
data['MixColor']=np.where(data.Color.str.contains('/'),'Mix','No Mix')
data['Color_new']=np.where(data.Color.str.contains('/'),data['Color'].str.split('/').str[0],data['Color'])
data.Color_new=data['Color_new'].str.split(' ').str[0]
more_removal = [i for i,x in dict(data['Color_new'].value_counts()).items() if x<100]
data['BaseColor']=np.where(data['Color_new'].isin(more_removal),'Other color',data['Color_new'])
data['BaseColor'].value_counts()

In [ ]:
data['Intake Condition']=data['Intake Condition'].replace(to_replace={'Behavior':'Other','Medical':'Injured'})

Color reduced to 25 bins. Split into two features - mix color and base color. 

Next steps:
- Clean out deprecated columns
- Engage in additional feature engineering

In [ ]:
data.rename(columns={'Mix':'MixBreed','Breed_test':'BaseBreed'},inplace=True) #standardize naming of columns
data=data[['Pet ID', 'Outcome Type', 'Sex upon Outcome', 'Name',
       'Found Location', 'Intake Type', 'Intake Condition', 'Pet Type',
       'Sex upon Intake', 'Age upon Intake Days',
       'Time at Center', 'MixBreed', 'BaseBreed','MixColor', 'BaseColor']]


<a id='sec-2'></a>
## 2. Feature Engineering

Ideas:
- Spay/Neuter status
- Spay/Neuter change status (at shelter)
- Location by City/State
- Name starting letter
- Name length


[*Back to top*](#top)

In [ ]:
data['NS_intake']=np.where((data['Sex upon Intake'].str.contains('Neutered'))
                          |(data['Sex upon Intake'].str.contains('Spayed')), 1,0) #create boolean infertility on intake
data['Gender']=data['Sex upon Intake'].str.split(' ').str[-1] #new feature

print(data.NS_intake.value_counts())
print(data.Gender.value_counts())

In [ ]:
data['Sex upon Outcome'].value_counts()

In [ ]:
data['NS_clinic']=np.where((data.NS_intake==0)&((data['Sex upon Outcome'].str.contains('Neutered'))
                          |(data['Sex upon Outcome'].str.contains('Spayed'))),1,0) #new feature
data['NS_clinic'].value_counts()

In [ ]:
data['Found Location']

We could input these into map api and get longitude and latitudinal data, or zip code and regionalize info.

We will just extract second last element in string (city)

In [ ]:
data['City']=data['Found Location'].str.split(' ').str[-2]
data.City.value_counts()

In [ ]:
bin_small('City','Other City',threshold=1000)
data.City.value_counts() #new feature

In [ ]:
data['FirstLetterName']=np.where(data.Name!='Unknown',data.Name.str[0],'Unknown')
data['FirstLetterName']=np.where(data.FirstLetterName.str.isalpha(),data.FirstLetterName,'ID')
data['FirstLetterName']=data.FirstLetterName.replace(to_replace={'É':'E'})
data['FirstLetterName'].fillna('Unknown',inplace=True)
data['FirstLetterName'].value_counts() #new feature

In [ ]:
data['NameLength']=data['Name'].map(lambda x: len(x))
data['NameLength'].value_counts()

In [ ]:
nl = []
for i in data.iterrows():
    if i[1].Name=='Unknown':
        nl.append('Unknown')
    else:
        if i[1].NameLength==0:
            nl.append('Unknown')
        elif (i[1].NameLength>=1)&(i[1].NameLength<4):
            nl.append('short')
        elif (i[1].NameLength>=1)&(i[1].NameLength<8):
            nl.append('med')
        else:
            nl.append('long')
data['NameLengthBin']=nl #new feature
data['NameLengthBin'].value_counts()

After adding 5 new features, let's remove depreciated columns again and move on to EDA and see if any of these additional features and original features hold any value

In [ ]:
#additional cleaning
bin_small('Outcome Type','Other Outcome',threshold=2000)
bin_small('Intake Condition','Other')
data['City'].fillna('Other City',inplace=True)

In [ ]:
data=data[['Pet ID', 'Outcome Type','Intake Type', 'Intake Condition', 'Pet Type','Age upon Intake Days', 'Time at Center',
       'MixBreed', 'BaseBreed', 'MixColor', 'BaseColor', 'NS_intake','NS_clinic', 'City', 'Gender', 'FirstLetterName','NameLengthBin']]

<a id='sec-3'></a>

## 3. Exploratory Data Analysis (EDA)
- Univariate analysis of dependent variable (Time at Center)
- Univariate analysis of discrete variable "Age upon Intake Days"
- Evaluate Age feature vs. Time at Center
- Chi-squared test of independence (Cat. vs Cat. vars)

[*Back to top*](#top)

### Univariate Analysis of "Time at Center" (Dependent variable, DV)

In [ ]:
data['Time at Center'].value_counts()

In [ ]:
data['Time at Center'].value_counts(normalize=True).plot(kind='pie',legend=True,autopct='%1.1f%%',figsize=(10,10))

Around 1:10 ratio between minority and majority class

### Age distribution

We're going to take another look at the distribution of the age variable. Earlier on we noticed high standard deviation, much larger than the mean. This suggests this variable has extreme upper bound outliers

In [ ]:
data['Age upon Intake Days'].describe()

In [ ]:
data['Age upon Intake Days'].skew()

In [ ]:
data['Age upon Intake Days'].kurtosis()

In [ ]:
ax = data['Age upon Intake Days'].plot(kind='hist',bins=20,title='Age Histogram 20 Bins')
ax.set_xlabel("Age (Days)")


In [ ]:
ax1 =data['Age upon Intake Days'].plot(kind='box',title='Age Box Plot')
ax1.set_ylabel('Frequency')

In [ ]:
data.loc[data['Age upon Intake Days']>2000].shape[0]

In [ ]:
data.loc[data['Age upon Intake Days']<=2000]['Age upon Intake Days'].plot(kind='box')
# s

Based on skew, kurtosis and graphs we can tell this variable is extremely non-normal. Even after removing the top 10% of age range, the boxplot still looks extremely skewed.

We have several options.

1. Remove extremely high age values 
    - 10% of dataset is over 2000 days in age
    - 25% of dataset is under 30 days in age
    - Different species have different longevity
2. Bin into ordinal categorical variable
3. Transform variable with log or boxcox transform

We will not remove the high values- they can still be relevant.

We will check the transformation outcomes and if that looks promising, we will use that approach. Otherwise we will bin to categorical values.

In [ ]:
data['Age_Shift1']=data['Age upon Intake Days']+1

In [ ]:
len(boxcox(data['Age_Shift1']))

In [ ]:
boxcox(data['Age_Shift1'])[0]

In [ ]:
boxcox(data['Age_Shift1'])[1] #lambda value of .1, close to log transformation

In [ ]:
data['AgeBoxCox']=boxcox(data['Age_Shift1'])[0]
ax2 = data['AgeBoxCox'].plot(kind='hist',bins=15, title='BoxCox Transform (Age) Historgram 15 Bin')
ax2.set_xlabel('BoxCox Transform of Age (λ=.1)')

In [ ]:
data.loc[(data.AgeBoxCox>=3)&(data.AgeBoxCox<6)]['Pet Type'].value_counts(normalize=True)

In [ ]:
data.loc[(data.AgeBoxCox>=3)&(data.AgeBoxCox<6)]['Time at Center'].value_counts()

In [ ]:
5055/(23620+5055)

In [ ]:
data.loc[(data.AgeBoxCox<=11)&(data.AgeBoxCox>6)]['Pet Type'].value_counts(normalize=True)

In [ ]:
data.loc[(data.AgeBoxCox<=11)&(data.AgeBoxCox>6)]['Time at Center'].value_counts()

In [ ]:
data.BaseBreed.value_counts()

In [ ]:
data['BaseBreed']=data['BaseBreed'].replace(to_replace={'fox':'Other','opossum':'Other','chicken':'Other','skunk':'Other','squirrel':'Other','Other german':'Other dog','Other pit':'Other dog','Other spaniel':'Other dog'})

Big difference in time spent at center vs. age just based on this investigation.

We will leave the age variable as is for now and revisit changing it as a feature later.

In [ ]:
cat_var = ['Outcome Type', 'Intake Type', 'Intake Condition',
       'Pet Type', 'MixBreed','BaseBreed', 'MixColor', 'BaseColor', 'NS_intake', 'NS_clinic',
       'City', 'Gender', 'FirstLetterName', 'NameLengthBin',]
min_class = data.loc[data['Time at Center']==1]
maj_class = data.loc[data['Time at Center']!=1]
cat={}
for i in cat_var:
    min_unq=min_class[i].unique()
    maj_unq=maj_class[i].unique()
    cat[i]= [i for i in maj_unq if i not in min_unq]
    print(i)
    print(cat[i])

### Chi-squared test of independence

In [ ]:
data['BaseBreed'].value_counts()

In [ ]:
cat_var = ['Outcome Type', 'Intake Type', 'Intake Condition',
       'Pet Type', 'MixBreed','BaseBreed', 'MixColor', 'BaseColor', 'NS_intake', 'NS_clinic',
       'City', 'Gender', 'FirstLetterName', 'NameLengthBin',]
min_class = data.loc[data['Time at Center']==1]
maj_class = data.loc[data['Time at Center']!=1]
for i in cat_var:
    min_class_dict = min_class[i].value_counts()
    min_dict=[x for i,x in sorted(min_class_dict.items(),key=lambda item:item[0])]
    maj_class_dict = maj_class[i].value_counts(normalize=True)*min_class.shape[0]
    maj_dict=[x for i,x in sorted(maj_class_dict.items(),key=lambda item:item[0])]
    print(f'Variable :{i}    |   {chisquare(min_dict,maj_dict)}')

City and MixBreed had highest pvalues.

In [ ]:
list(min_class.NS_intake.value_counts())[-1]

In [ ]:
#most frequent bin
#% of most frequent bin
#Number of bins
#smallest bin
#small bin % frequency
col = ['Column']
data_in=[]
for i in cat_var:
    for count,df in enumerate([data,min_class,maj_class]):
        if count==0:
            pref = 'All'
        elif count==1:
            pref = 'Min'
        elif count==2:
            pref='Maj'
        row = [i,pref]
        val_counts = df[i].value_counts(normalize=True)
        mfb=val_counts.index[0]
        pmfb=val_counts[0]
        row.append(mfb)
        row.append(round(pmfb*100,1))
        row.append(len(val_counts))
        row.append(val_counts.index[-1])
        row.append(round(list(val_counts)[-1]*100,1))
        data_in.append(row)
col.append('Class')
col.append('Mode')
col.append('Mode%')
col.append('BinNumber')
col.append('Min')
col.append('Min%')
cat_summ = pd.DataFrame(data=data_in,columns=col)
cat_summ

In [ ]:
all_df = data['Outcome Type'].value_counts().to_frame()
all_df

In [ ]:
from ipywidgets import interact
@interact(cat =['Outcome Type', 'Intake Type', 'Intake Condition',
       'Pet Type', 'MixBreed','BaseBreed', 'MixColor', 'BaseColor', 'NS_intake', 'NS_clinic',
       'City', 'Gender', 'FirstLetterName', 'NameLengthBin',])
def show_freq(cat):
    plt.figure(figsize=(12, 8), dpi=80)
    all_df = pd.DataFrame()
    for count,df in enumerate([data,min_class,maj_class]):
        if count==0:
            pref = 'All'
        elif count==1:
            pref = 'Min'
        elif count==2:
            pref='Maj'
        ndf = df[cat].value_counts(normalize=True).to_frame()
        ndf.rename(columns={cat:'Count'},inplace=True)
        ndf['Class']=pref
        all_df=all_df.append(ndf)
    all_df[cat]=all_df.index
    ax = sns.barplot(x=cat, y="Count", hue="Class", data=all_df)
    plt.show()

In [ ]:
data=data[['Pet ID', 'Outcome Type', 'Intake Type', 'Intake Condition',
       'Pet Type',  'Time at Center', 'MixBreed',
       'BaseBreed', 'MixColor', 'BaseColor', 'NS_intake', 'NS_clinic',
       'City', 'Gender', 'FirstLetterName', 'NameLengthBin', 'Age_Shift1']]
cat =['Outcome Type', 'Intake Type', 'Intake Condition',
'Pet Type', 'MixBreed','BaseBreed', 'MixColor', 'BaseColor', 'NS_intake', 'NS_clinic',
'City', 'Gender', 'FirstLetterName', 'NameLengthBin',]
data=pd.get_dummies(data,drop_first=True,columns=cat) #one hot encode categorical variables

In [ ]:
data.to_csv(r'..\data\feature_final.csv',index=False)

In [ ]:
legacy_data

# END

[*Back to top*](#top)